In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons
# import uniform distribution
from scipy.stats import uniform
from scipy.stats import binom
from scipy.stats import norm
import scipy.special as sc
import scipy.optimize
from scipy.optimize import fsolve
from scipy.stats import pareto
import time
np.random.seed(5)

In [2]:
def auction_exp(data_auction, p):
    """auction experiment simulation

    Args:
        data_auction: random sample data for auction simulation
        p: the reserve price
        
    Return:
        revenue: the expected revenue
    """
    revenue = 0
    for bid in data_auction:
        if bid >= p:
          #change the bidder with price p
          revenue += p
    
    revenue = revenue/len(data_auction)
    return revenue

In [3]:
def rejection_sampling(pi1, pi2, N):
    """rejection sampling algorithm

    Args:
        pi1,pi2: the range of log-lottery distribution
        N: the number of samples from the log-lottery distribution
        
    Return:
        rej_sample: The random samples from the log-lottery distribution
    """

    #using rejection sampling, get N random price samples
    n = 0
    don = pi2*np.log(pi2/pi1) - (pi2-pi1)
    env =(pi2/pi1-1)/don
    rej_sample =[]
    while(n <= N):
        x = uniform.rvs(loc=pi1, scale=pi2-pi1)
        #a is acception probability
        a = (pi2/x-1)/don/env
        u1 = uniform.rvs()
        if (u1 <= a):
            rej_sample.append(x)
            n += 1
    return rej_sample

In [7]:
# Experiment deterministic and randomized auctions for single-item and single-bidder case with Pareto(1) distribution:
#Test performance of numertical solver and thus adjust the initial starting point############################\

#Definition 3.1: rho_D function used to compute DAPX
def rho_D(x):
    return ((x-1)**3)/((2*x-1)**2) - r**2   

#Definition 5.1:  rho function used to compute the APX theoretical upper bound
def random_rho(x):
    return ((2*np.exp(x-1)-1)-(x**2)*r**2 - x**2)

#Function f used to solve pi1 for log-lottery distribution
def f(x):
    return (x * (1 + np.log(k2/(x**2) + 1) - np.log(2)) -u )
 
n = 100000
step = np.linspace(0.001,10,50)

for i in step:
    r = i
    c = 1+np.sqrt(1+1/(r**2))
    u, var = pareto.stats(c, moments='mv')
    sigma = np.sqrt(var)
    r = sigma/u
  
    


    #use numerical solver to solve the ratio value, then compute the reserve price
    p_d = fsolve(rho_D, [u-0.1])
    print(rho_D(p_d))
   
    #First solve pi1 and pi2 using numerical solver and then generate the samples from 
    #the corresponding log-lottery distribution.
    k2 = u**2 + sigma**2
    pi1 = fsolve(f, [u-0.1])
    print(f(pi1))
   
     
    #compute the APX theoretical upper bound using function rho from Definition 4.1
    r_rho = fsolve(random_rho, [u+10])
    print(random_rho(r_rho))


[5.15569897e-17]
[0.]
[-2.22044605e-16]
[2.08166817e-17]
[4.4408921e-16]
[2.22044605e-15]
[-2.77555756e-16]
[1.26565425e-14]
[5.68434189e-14]
[-8.8817842e-16]
[2.22044605e-16]
[5.32907052e-15]
[0.]
[0.]
[4.26325641e-14]
[0.]
[2.22044605e-16]
[2.50111043e-12]
[2.22044605e-16]
[0.]
[-3.55271368e-15]
[-3.10862447e-15]
[2.22044605e-16]
[1.95399252e-13]
[-8.8817842e-16]
[6.66133815e-16]
[1.77635684e-14]
[4.4408921e-16]
[2.44249065e-15]
[5.47117907e-13]
[0.]
[3.99680289e-15]
[-4.26325641e-14]
[-8.8817842e-16]
[3.55271368e-15]
[1.9120705e-11]
[-8.8817842e-16]
[1.99840144e-15]
[1.3500312e-13]
[-8.8817842e-16]
[1.11022302e-15]
[-7.81597009e-14]
[0.]
[6.66133815e-16]
[5.7610805e-11]
[-1.77635684e-15]
[2.22044605e-16]
[1.07291953e-12]
[0.]
[0.]
[-1.3500312e-13]
[1.77635684e-15]
[0.]
[2.84217094e-14]
[0.]
[0.]
[9.7131192e-11]
[-1.77635684e-15]
[2.22044605e-16]
[4.12114787e-12]
[0.]
[0.]
[1.49213975e-13]
[0.]
[-2.22044605e-16]
[-5.68434189e-14]
[3.55271368e-15]
[0.]
[2.55795385e-13]
[-3.55271368e-1

In [8]:
r_rho

array([9.40445157])

In [9]:
# Experiment deterministic and randomized auctions for single-item and single-bidder case with Pareto(1) distribution:
sigma_list = []
DAPX_pareto1 = []
r_list =[]
p_reserve_list = []
p_d_list = []
c_list=[]
r_rho_list=[]
APX_pareto1 = []

#Definition 3.1: rho_D function used to compute DAPX
def rho_D(x):
    return ((x-1)**3)/((2*x-1)**2) - r**2   

#Definition 5.1:  rho function used to compute the APX theoretical upper bound
def random_rho(x):
    return ((2*np.exp(x-1)-1)-(x**2)*r**2 - x**2)

#Function f used to solve pi1 for log-lottery distribution
def f(x):
    return (x * (1 + np.log(k2/(x**2) + 1) - np.log(2)) -u )
 
n = 100000
step = np.linspace(0.001,10,50)
tic = time.clock()
for i in step:
    r = i
    c = 1+np.sqrt(1+1/(r**2))
    c_list.append(c)
    u, var = pareto.stats(c, moments='mv')
    sigma = np.sqrt(var)
    r = sigma/u
    r_list.append(r)
    
    bid_pareto= pareto.rvs(c,size=n)

    #use numerical solver to solve the ratio value, then compute the reserve price
    p_d = fsolve(rho_D, [u-0.1])
    print(rho_D(p_d))
    p_d_list.append(p_d)
    p_reserve = p_d*u/(2*p_d-1)
    p_reserve_list.append(p_reserve)
    #Expertiment Giannakopoulos et al. auction
    exp_revenue = auction_exp(bid_pareto,p_reserve)
    ###############################################################################
    #For Pareto(1) the optimal revenue is 1
    opt = 1
    #Compute the experimental DAPX
    DAPX_pareto1.append((opt/exp_revenue)[0])  
    ###############################################################################
    
    #First solve pi1 and pi2 using numerical solver and then generate the samples from 
    #the corresponding log-lottery distribution.
    k2 = u**2 + sigma**2
    pi1 = fsolve(f, [u-0.1])
    pi2 = 0.5*(k2/pi1 +pi1)
    print(f(pi1))
    rej_sample = []
    rej_sample = rejection_sampling(pi1,pi2,10000)
    rexp_revenue_list = []
    #For each reserve price, perform auction experiment and save all the results
    for rp_reserve in rej_sample:
        rexp_revenue_list.append(auction_exp(bid_pareto,rp_reserve))
    #Determine the experimental APX with optimal revenue against the average revenue
    APX_pareto1.append(opt/np.average(rexp_revenue_list))
     
    #compute the APX theoretical upper bound using function rho from Definition 4.1
    r_rho = fsolve(random_rho, [u+10])
    r_rho_list.append(r_rho[0])
    print(random_rho(r_rho))


toc = time.clock()
print('time: ' + str(toc-tic))

C:\Users\Ariel Lou\anaconda3\lib\site-packages\ipykernel_launcher.py:25: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


[5.15569897e-17]
[0.]
[-1.001535e-06]
[2.08166817e-17]
[4.4408921e-16]


KeyboardInterrupt: 

In [8]:
# Experiment Randomisation first compute the log-lottery with Pareto(1) distribution:
sigma_list = []
DAPX_pareto = []
r_list =[]
reserve_p_list = []
p_d_list = []
c_list=[]

APX_pareto = []

n = 100000
step = np.linspace(0.001,10,50)
def F(x):
    return ((x-1)**3)/((2*x-1)**2) - r**2   

#theorem 2 
def random_rho(x):
    return ((2*np.exp(x-1)-1)-(x**2)*r**2 - x**2)
def f(x):
    return (x * (1 + np.log(k2/(x**2) + 1) - np.log(2)) -u )
 
tic = time.clock()
for i in step:
    r = i
    c = 1+np.sqrt(1+1/(r**2))
    c_list.append(c)
    u, var = pareto.stats(c, moments='mv')
    sigma = np.sqrt(var)
    bid_pareto= pareto.rvs(c,size=n)

    r = sigma/u
    r_list.append(r)

    #p_d = 1 + 4* (sigma/u)**2
    p_d = fsolve(F, [u-0.1])
    print(F(p_d))
  
    k2 = u**2 + sigma**2
    pi1 = fsolve(f, [u-0.1])
    pi2 = 0.5*(k2/pi1 +pi1)
    print(f(pi1))
   
     
    #compute the APX using full-bundle log lottery randomization 
    r_rho = fsolve(random_rho, [u])

    print(random_rho(r_rho))


toc = time.clock()
print('time: ' + str(toc-tic))

C:\Users\Ariel Lou\anaconda3\lib\site-packages\ipykernel_launcher.py:22: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
C:\Users\Ariel Lou\anaconda3\lib\site-packages\scipy\optimize\minpack.py:162: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
C:\Users\Ariel Lou\anaconda3\lib\site-packages\scipy\optimize\minpack.py:162: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)


[5.15569897e-17]
[0.]
[-1.001535e-06]
[2.08166817e-17]
[4.4408921e-16]
[-3.33066907e-14]
[-2.77555756e-16]
[1.26565425e-14]
[-0.07783956]
[-8.8817842e-16]
[2.22044605e-16]
[-0.12253277]
[0.]
[0.]
[-0.15692722]
[0.]
[2.22044605e-16]
[-0.18203755]
[2.22044605e-16]
[0.]
[-0.2001392]
[-3.10862447e-15]
[2.22044605e-16]
[-0.21332065]
[-8.8817842e-16]
[6.66133815e-16]
[-0.22305916]
[4.4408921e-16]
[2.44249065e-15]
[-0.23038322]
[0.]
[3.99680289e-15]
[-0.23599151]
[-8.8817842e-16]
[3.55271368e-15]
[-0.24041786]
[-8.8817842e-16]
[1.99840144e-15]
[-0.24381753]
[-8.8817842e-16]
[1.11022302e-15]
[-0.24659362]
[0.]
[6.66133815e-16]
[-0.24885304]
[-1.77635684e-15]
[2.22044605e-16]
[-0.25071153]
[0.]
[0.]
[-0.25225895]
[1.77635684e-15]
[0.]
[-0.25371382]
[0.]
[0.]
[-0.25466187]
[-1.77635684e-15]
[2.22044605e-16]
[-0.2556042]
[0.]
[0.]
[-0.25641559]
[0.]
[-2.22044605e-16]
[-0.25711896]
[3.55271368e-15]
[0.]
[-0.25773248]
[-3.55271368e-15]
[2.22044605e-16]
[-0.25827069]
[0.]
[0.]
[-0.25874533]
[3.55271

C:\Users\Ariel Lou\anaconda3\lib\site-packages\ipykernel_launcher.py:50: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


In [ ]:
# create figure and axis objects with subplots()
fig,ax = plt.subplots(figsize=(20, 10))
ax.tick_params(axis='x', labelsize= 16)
ax.tick_params(axis='y', labelsize= 16)

# set x-axis label
ax.set_xlabel("r = \u03C3/u",fontsize=20)
# set y-axis label
ax.set_ylabel("DAPX_Pareto(1)",fontsize=20)

ax.plot(r_list, DAPX_pareto1,color="green",marker="o",label="Experimental DAPX")
#ax.plot(r_list, p_d_list,color="blue",marker="o",label="Theoritical \u03C1_D")

ax.set_ylim(1,1.16)
plt.legend(loc="upper left",fontsize = 20)
#plt.savefig("pareto1_dapx_detail.png",bbox_inches='tight')
plt.show()

In [38]:
plt.figure(figsize=(15,10))
plt.tick_params(axis='x', labelsize= 16)
plt.tick_params(axis='y', labelsize= 16)
plt.xlabel("r = \u03C3/u",fontsize=16)
plt.ylabel("OPT/REV",fontsize=16)
plt.plot(r_list, APX_pareto1,color="#FFA500",label="Experimental APX")
plt.plot(r_list, DAPX_pareto1,color="green",label="Experimental DAPX")

lower_bound = 1+np.log(1+np.power(r_list,2))
#plt.plot(r_list, r_rho,color="blue",label="Theoretical APX upper bound \u03C1")
plt.plot(r_list, lower_bound,color="red",label="Theoretical APX lower bound 1+ln(1+r^2)")
plt.legend(loc="upper left",fontsize = 16)
#plt.savefig("apxpareto1.png",bbox_inches='tight')
plt.show()

In [9]:
import pandas as pd 
  
# creating the DataFrame 
my_df = { 'r' : r_list,
          'DAPX_pareto': DAPX_pareto,
         'theoretical DAPX':p_d_list,
         'APX_pareto': APX_pareto,
         'upper_bound':r_rho_list,
         'lower_bound':lower_bound
         } 
df = pd.DataFrame(my_df) 
  
   
# saving the DataFrame as a CSV file 
gfg_csv_data = df.to_csv('apx_pareto1.csv', index = True) 
print('\nCSV String:\n', gfg_csv_data) 


CSV String:
 None
